## Neo4j Vector Chain

In [1]:
import dotenv

In [2]:
dotenv.load_dotenv()

True

In [3]:
from chatbot_api.src.chains.hospital_review_chain import (
    reviews_vector_chain
)

In [4]:
query = """What have patients said about hospital efficiency?
Mention details from specific revies."""

response = reviews_vector_chain.invoke(query)

In [5]:
response.get("result")

'Patients have mentioned both positive and negative aspects of hospital efficiency in their reviews.\n\n- Beverly Johnson mentioned that the hospital had a modern feel, and the staff was attentive at Brown Inc. However, she found the bureaucratic procedures for check-in and discharge to be cumbersome.\n- Anna Green praised the courteous staff and impressive cleanliness standards at Burch-White. However, she noted that the outdated magazines in the waiting area did little to alleviate boredom during long waits.\n- Michael Johnson had a mix of positive and negative experiences at Garcia Ltd. While he found the medical care to be competent, he mentioned that the administrative processes were confusing, leading to unnecessary stress.\n- Jesse Marquez highlighted the excellent medical care provided at Wallace-Hamilton. He also mentioned that the nursing staff was attentive. However, he found the administrative processes to be cumbersome and suggested they could be streamlined for efficiency

# Cypher Chain

In [3]:
import dotenv

dotenv.load_dotenv()

True

In [4]:
from chatbot_api.src.chains.hospital_cypher_chain import (
    hospital_cypher_chain
)

question = """What is the average visit duration for \
emergency visits in North Carolina?"""

response = hospital_cypher_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (v:Visit)-[:AT]->(h:Hospital)
WHERE h.state_name = 'NC' AND v.admission_type = 'Emergency' AND v.status = 'DISCHARGED'
WITH v, duration.between(date(v.admission_date), date(v.discharge_date)).days AS visit_duration
RETURN AVG(visit_duration) AS average_visit_duration
Full Context:
[{'average_visit_duration': 15.072972972972991}]

> Finished chain.


In [5]:
response.get("result")

'The average visit duration for emergency visits in North Carolina is approximately 15.07 days.'

In [7]:
question = """Which state had the largest percent increase \
in medicaid visits from 2022 to 2023?"""

response = hospital_cypher_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (h:Hospital)<-[:AT]-(v:Visit)-[:COVERED_BY]->(p:Payer)
WHERE p.name = 'Medicaid' AND v.admission_date >= '2022-01-01' AND
v.admission_date < '2024-01-01'
WITH h.state_name AS state, COUNT(v) AS visit_count,
     SUM(CASE WHEN v.admission_date >= '2022-01-01' AND
     v.admission_date < '2023-01-01' THEN 1 ELSE 0 END) AS count_2022,
     SUM(CASE WHEN v.admission_date >= '2023-01-01' AND
     v.admission_date < '2024-01-01' THEN 1 ELSE 0 END) AS count_2023
WITH state, visit_count, count_2022, count_2023,
     (toFloat(count_2023) - toFloat(count_2022)) / toFloat(count_2022) * 100
     AS percent_increase
RETURN state, percent_increase
ORDER BY percent_increase DESC
LIMIT 1
Full Context:
[{'state': 'TX', 'percent_increase': 8.823529411764707}]

> Finished chain.


In [8]:
response.get("result")

'The state with the largest percent increase in Medicaid visits from 2022 to 2023 is Texas (TX) with a percent increase of 8.82%.'